In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import RepeatedStratifiedKFold, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV

stats_df = pd.read_csv(stats_df, sep = "\t")

train_df = stats_df.iloc[:70,2:len(stats_df.columns)]
train_score = stats_df.iloc[:70,0]

test_df = stats_df.iloc[70:91,2:len(stats_df.columns)]
test_score = stats_df.iloc[70:91,0]

print(test_df)
print(test_class)

In [ ]:
from sklearn.neural_network import MLPClassifier

parameters = {'solver':['lbfgs'], 'activation':['identity', 'logistic', 'tanh', 'relu'], 'alpha':[.1, .01, .001], 'max_iter':[500]}
ml = MLPClassifier()
ml.fit(train_df, train_class)
predict_df = ml.predict(test_df)
probability_df = ml.predict_proba(test_df)
ml_score = ml.score(test_df, test_class)

print()
print()
print()

In [ ]:
from sklearn.linear_model import LogisticRegression
parameters = {'multi_class':['ovr', 'multinomial'], 'penalty':['l2'], 'C':[.1, 1.0, 10], 'solver':['lbfgs', 'newton-cg'], 'class_weight':['balanced'], 'max_iter':[500]}
ml = LogisticRegression() #the model we are running
clf.fit(train_df, train_class) #fits the best parameters to the training data
predict_df = clf.predict(test_df) #predicts the classes on the test df
probability_df = clf.predict_proba(test_df) #gets the probabilites of each instance belonging to the class on the test df
ml_score = clf.score(test_df, test_class) #calculates the score of correct classifications from predict

print()
print()
print()